In [461]:
import pandas as pd
import numpy as np

from cryptodatapy.extract.datarequest import DataRequest
from cryptodatapy.util.datacatalog import DataCatalog 
from cryptodatapy.extract.getdata import GetData
from cryptodatapy.transform.od import OutlierDetection
from cryptodatapy.transform.impute import Impute
from cryptodatapy.transform.filter import Filter
from cryptodatapy.transform.clean import CleanData, stitch_dataframes
from cryptodatapy.transform.impute import Impute

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


# Data

In [2]:
dc = DataCatalog()

In [24]:
us_rates_tickers = dc.get_tickers_metadata(cat='rates', subcat='yield', mkt_type='spot').loc[:, 'fred_id'].dropna().index.tolist()

In [450]:
data_req = DataRequest(source='yahoo', source_tickers=['^MOVE'], cat='rates')

In [452]:
move_df = GetData(data_req).get_series()

[*********************100%***********************]  1 of 1 completed


In [456]:
move_df = move_df.rename(index={'^MOVE': 'US_Rates_Vol'})

In [27]:
data_req = DataRequest(source='fred', tickers=us_rates_tickers, cat='rates')

In [28]:
us_rates_df = GetData(data_req).get_series()

In [457]:
us_rates_df.unstack().close.columns

Index(['US_Rates_10Y', 'US_Rates_1M', 'US_Rates_1Y', 'US_Rates_20Y',
       'US_Rates_2Y', 'US_Rates_30Y', 'US_Rates_3M', 'US_Rates_3Y',
       'US_Rates_5Y', 'US_Rates_6M', 'US_Rates_7Y'],
      dtype='object', name='ticker')

In [458]:
us_rates_df = pd.concat([us_rates_df, move_df]).sort_index()

In [31]:
# us_rates_df.to_csv('us_rates_daily.csv')

In [460]:
us_rates_df.to_parquet('s3://factorlab-data/us_yields_daily.parquet')

In [428]:
er_df = pd.read_csv('../../../../rose_excess_returns_daily.csv', index_col=['date'], parse_dates=['date'])

In [429]:
prices_df = pd.read_csv('../../../../rose_asset_prices_daily.csv', index_col=['date'], parse_dates=['date'])

In [430]:
iso3_to_iso2 = {
     'arg': 'AR',
     'aus': 'AU',
     'aut': 'AT',
     'bel': 'BE',
     'bra': 'BR',
     'can': 'CA',
     'che': 'CH',
     'chl': 'CL',
     'chn': 'CN',
     'col': 'CO',
     'cze': 'CZ',
     'deu': 'DE',
     'dnk': 'DK', 
     'esp': 'ES',
     'eur': 'EU',
     'fin': 'FI',
     'fra': 'FR',
     'gbr': 'GB',
     'grc': 'GR',
     'hkg': 'HK',
     'hun': 'HU',
     'idn': 'ID',
     'ind': 'IN',
     'irl': 'IE',
     'ita': 'IT',
     'isl': 'IS',
     'isr': 'IL',
     'jpn': 'JP',
     'mex': 'MX',
     'mys': 'MY',
     'nld': 'NL',
     'nor': 'NO',
     'nzl': 'NZ',
     'per': 'PE',
     'phl': 'PH',
     'pol': 'PL',
     'prt': 'PT',
     'rus': 'RU',
     'sgp': 'SG',
     'swe': 'SE',
     'kor': 'KR',
     'swe': 'SE',
     'twn': 'TW',
     'tha': 'TH',
     'tur': 'TR',
     'usa': 'US',
     'zaf': 'ZA'
}

In [431]:
asset_mapping = {
    'lr': 'Rates_Long', 
    'sr': 'Rates_Short', 
    'eq': 'Eqty',
    'fx': 'FX'
}

In [432]:
def map_columns(col_name):
    country_code, asset_code = col_name.split('.')
    new_country = iso3_to_iso2.get(country_code, country_code)  # Handle missing mappings gracefully
    new_asset = asset_mapping.get(asset_code, asset_code)  # Handle missing mappings gracefully
    
    if new_asset:  # Only add underscore if the asset name is not empty
        return f'{new_country}_{new_asset}'
    else:
        return new_country

In [433]:
def map_columns_ret(col_name):
    country_code, asset_code, ret = col_name.split('.')
    new_country = iso3_to_iso2.get(country_code, country_code)  # Handle missing mappings gracefully
    new_asset = asset_mapping.get(asset_code, asset_code)  # Handle missing mappings gracefully
    
    if new_asset:  # Only add underscore if the asset name is not empty
        return f'{new_country}_{new_asset}'
    else:
        return new_country

In [434]:
# asset dfs
eqty_df = prices_df.loc[:, prices_df.columns[:41]]
eqty_er_df = er_df.loc[:, er_df.columns[:41]]
rates_df = prices_df.loc[:, prices_df.columns[41:123]]
rates_er_df = er_df.loc[:, er_df.columns[42:83]]
fx_df = prices_df.loc[:, prices_df.columns[123:164]]
fx_er_df = er_df.loc[:, er_df.columns[83:124]]
cmdty_df = prices_df.loc[:, prices_df.columns[164:]]
cmdty_er_df = er_df.loc[:, er_df.columns[124:]]

In [435]:
# convert price cols
eqty_df.columns = [map_columns(col) for col in eqty_df.columns]
rates_df.columns = [map_columns(col) for col in rates_df.columns]
fx_df.columns = [map_columns(col) for col in fx_df.columns]
cmdty_df.columns = [col.title() for col in cmdty_df.columns]
cmdty_df.columns = ['Gold', 'Silver', 'Platinum', 'Palladium', 'Oil_WTI', 'Oil_Brent', 'Natgas',
       'Rbob', 'Corn', 'Wheat', 'Soybeans', 'Coffee', 'Sugar', 'Cotton',
       'Orange_Juice', 'Cocoa', 'Pork', 'Beef', 'Aluminum', 'Copper', 'Lead',
       'Nickel', 'Tin', 'Zinc']

In [436]:
# convert ret cols
eqty_er_df.columns = [map_columns_ret(col) for col in eqty_er_df.columns]
rates_er_df.columns = [map_columns_ret(col) for col in rates_er_df.columns]
fx_er_df.columns = [map_columns_ret(col) for col in fx_er_df.columns]
cmdty_er_df.columns = [col.title().split('.')[0] for col in cmdty_er_df.columns]
cmdty_er_df.columns = ['Gold', 'Silver', 'Platinum', 'Palladium', 'Oil_WTI', 'Oil_Brent', 'Natgas',
       'Rbob', 'Corn', 'Wheat', 'Soybeans', 'Coffee', 'Sugar', 'Cotton',
       'Orange_Juice', 'Cocoa', 'Pork', 'Beef', 'Aluminum', 'Copper', 'Lead',
       'Nickel', 'Tin', 'Zinc']

In [437]:
# stack dfs
eqty_df = eqty_df.stack().to_frame('close')
eqty_er_df = eqty_er_df.stack().to_frame('er')
rates_df = rates_df.stack().to_frame('close')
rates_er_df = rates_er_df.stack().to_frame('er')
fx_df = fx_df.stack().to_frame('close')
fx_er_df = fx_er_df.stack().to_frame('er')
cmdty_df = cmdty_df.stack().to_frame('close')
cmdty_er_df = cmdty_er_df.stack().to_frame('er')

In [438]:
# set indexes
eqty_df.index.names = ['date', 'ticker']
eqty_er_df.index.names = ['date', 'ticker']
rates_df.index.names = ['date', 'ticker']
rates_er_df.index.names = ['date', 'ticker']
fx_df.index.names = ['date', 'ticker']
fx_er_df.index.names = ['date', 'ticker']
cmdty_df.index.names = ['date', 'ticker']
cmdty_er_df.index.names = ['date', 'ticker']

In [439]:
eqty_df = pd.concat([eqty_df, eqty_er_df], axis=1).sort_index()
rates_df = pd.concat([rates_df, rates_er_df], axis=1).sort_index()
fx_df = pd.concat([fx_df, fx_er_df], axis=1).sort_index()
cmdty_df = pd.concat([cmdty_df, cmdty_er_df], axis=1).sort_index()

In [441]:
eqty_df.to_parquet('s3://factorlab-data/global_eqty_data_daily.parquet')

In [442]:
rates_df.to_parquet('s3://factorlab-data/global_rates_data_daily.parquet')

In [443]:
fx_df.to_parquet('s3://factorlab-data/global_fx_data_daily.parquet')

In [444]:
cmdty_df.to_parquet('s3://factorlab-data/commodity_data_daily.parquet')